In [1]:
%pip install -q html2text llama-index pandas pyarrow tqdm
%pip install -q llama-index-readers-web
%pip install -q llama-index-callbacks-openinference

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
%pip install arize-phoenix


Note: you may need to restart the kernel to use updated packages.


In [3]:
import os
from llama_index.core import Settings
from llama_index.llms.ollama import Ollama
from llama_index.embeddings.ollama import OllamaEmbedding

# Configure Ollama LLM
ollama_llm = Ollama(
    #model="llama3.2:latest",
    model="mistral:7b",
    base_url="http://localhost:11434",
    temperature=0.1
)

# Configure embedding model
ollama_embedding = OllamaEmbedding(
    model_name="nomic-embed-text:latest",
    base_url="http://localhost:11434",
    ollama_additional_kwargs={"mirostat": 0}
)

Settings.llm = ollama_llm
Settings.embed_model = ollama_embedding

In [4]:
import textwrap

import llama_index.core
from llama_index.core import VectorStoreIndex
from llama_index.core.node_parser import SentenceSplitter
from llama_index.core.callbacks import CallbackManager
from llama_index.callbacks.openinference import OpenInferenceCallbackHandler
from llama_index.callbacks.openinference.base import (
    as_dataframe,
)


In [5]:
import os
from getpass import getpass

# Add Phoenix API Key for tracing

if not (phoenix_api_key := os.getenv("PHOENIX_API_KEY")):
    phoenix_api_key = getpass("🔑 Enter your Phoenix API key: ")

os.environ["PHOENIX_API_KEY"] = phoenix_api_key

In [6]:
# setup Arize Phoenix for logging/observability
import os

from openinference.instrumentation.llama_index import LlamaIndexInstrumentor

from phoenix.otel import register

os.environ["PHOENIX_CLIENT_HEADERS"] = f"api_key={os.environ['PHOENIX_API_KEY']}"
os.environ["PHOENIX_COLLECTOR_ENDPOINT"] = "https://app.phoenix.arize.com"

# Configuration is picked up from your environment variables
tracer_provider = register(project_name="my-custom-project")

# Instrument LlamaIndex. This allows Phoenix to collect traces from LlamaIndex queries.
LlamaIndexInstrumentor().instrument(tracer_provider=tracer_provider, skip_dep_check=True)

OpenTelemetry Tracing Details
|  Phoenix Project: my-custom-project
|  Span Processor: SimpleSpanProcessor
|  Collector Endpoint: https://app.phoenix.arize.com/v1/traces
|  Transport: HTTP
|  Transport Headers: {'api_key': '****', 'authorization': '****'}
|  
|  Using a default SpanProcessor. `add_span_processor` will overwrite this default.
|  
|  `register` has set this TracerProvider as the global OpenTelemetry default.
|  To disable this behavior, call `register` with `set_global_tracer_provider=False`.



In [7]:
from llama_index.core import SimpleDirectoryReader

#documents = SimpleDirectoryReader(input_files=['../data/2022 Q3 AAPL.pdf']).load_data()
documents = SimpleDirectoryReader(input_files=['../data/paul_graham_essay3.txt']).load_data()

In [8]:
parser = SentenceSplitter()
nodes = parser.get_nodes_from_documents(documents)
print(nodes[0].text)

What I Worked On

February 2021

Before college the two main things I worked on, outside of school, were writing and programming. I didn't write essays. I wrote what beginning writers were supposed to write then, and probably still are: short stories. My stories were awful. They had hardly any plot, just characters with strong feelings, which I imagined made them deep.

The first programs I tried writing were on the IBM 1401 that our school district used for what was then called "data processing." This was in 9th grade, so I was 13 or 14. The school district's 1401 happened to be in the basement of our junior high school, and my friend Rich Draves and I got permission to use it. It was like a mini Bond villain's lair down there, with all these alien-looking machines — CPU, disk drives, printer, card reader — sitting up on a raised floor under bright fluorescent lights.

The language we used was an early version of Fortran. You had to type programs on punch cards, then stack them in the

In [9]:
callback_handler = OpenInferenceCallbackHandler()
callback_manager = CallbackManager([callback_handler])
llama_index.core.Settings.callback_manager = callback_manager

In [10]:
index = VectorStoreIndex.from_documents(documents)
query_engine = index.as_query_engine()

In [11]:
max_characters_per_line = 80
queries = [
    "What did Paul Graham do growing up?",
    "When and how did Paul Graham's mother die?",
    "What, in Paul Graham's opinion, is the most distinctive thing about YC?",
    "When and how did Paul Graham meet Jessica Livingston?",
    "What is Bel, and when and where was it written?",
]
for query in queries:
    response = query_engine.query(query)
    print("Query")
    print("=====")
    print(textwrap.fill(query, max_characters_per_line))
    print()
    print("Response")
    print("========")
    print(textwrap.fill(str(response), max_characters_per_line))
    print()

Query
=====
What did Paul Graham do growing up?

Response
 Growing up, Paul Graham was primarily interested in two areas outside of
school: writing and programming. He wrote short stories but found them to be
lacking in plot. His first experience with programming was using an early
version of Fortran on the IBM 1401 computer in his junior high school's data
processing center. However, he didn't write any memorable programs due to the
limitations of the machine and lack of data storage.     It wasn't until
microcomputers became available that everything changed for Paul. He was
impressed by a friend who built a Heathkit computer and watched him typing
programs directly into it. After years of nagging, his father bought him a
TRS-80 microcomputer in about 1980, which he used to write simple games, predict
the flight trajectory of model rockets, and create a word processor for his
father to use.     Despite his interest in programming, Paul initially planned
to study philosophy in college

In [12]:
query_data_buffer = callback_handler.flush_query_data_buffer()
query_dataframe = as_dataframe(query_data_buffer)
query_dataframe

,:id.id:,:timestamp.iso_8601:,:feature.text:prompt,:feature.[float].embedding:prompt,:feature.text:llm_prompt,:feature.[str]:llm_messages,:prediction.text:response,:feature.[str].retrieved_document_ids:prompt,:feature.[float].retrieved_document_scores:prompt
0,1eade646-f4c7-4bcf-8c7a-e52db222d8cf,2024-12-23T13:33:06.363527,What did Paul Graham do growing up?,"[0.06617288291454315, 1.0513052940368652, -3.8...",None,"[(system, You are an expert Q&A system that is...","Growing up, Paul Graham was primarily interes...",[b6f6b4daf8a1f32f7d4dd3032ef0c7b0b362c8c2481eb...,"[0.4677078852797752, 0.4495614081785316]"
1,9c9e25a4-8689-48c5-af91-375ccaec015b,2024-12-23T13:33:21.257303,When and how did Paul Graham's mother die?,"[1.5763593912124634, 0.10003548860549927, -3.0...",None,"[(system, You are an expert Q&A system that is...",The provided context does not contain any inf...,[0d6569b4324b00bb32b570334bfc26f45528dd6d9f480...,"[0.4072749482713961, 0.39325457483605925]"
2,52f04e71-b421-4b93-8813-4a6fdfa14f20,2024-12-23T13:33:30.889343,"What, in Paul Graham's opinion, is the most di...","[0.12026181071996689, 0.3832641541957855, -3.9...",None,"[(system, You are an expert Q&A system that is...","In Paul Graham's opinion, the most distinctiv...",[5c63eb641a15879f19178420b4749812c0fe88aec37e3...,"[0.4705470043798403, 0.42478806306184663]"
3,6d2a5064-d730-4e79-9c76-a25552c5a8c8,2024-12-23T13:33:41.222969,When and how did Paul Graham meet Jessica Livi...,"[0.3744025230407715, 1.1693042516708374, -3.20...",None,"[(system, You are an expert Q&A system that is...",The provided context does not contain any inf...,[0d6569b4324b00bb32b570334bfc26f45528dd6d9f480...,"[0.39885110168131893, 0.3921428054521445]"
4,b8c49a1f-c979-4eec-8f4a-e1ad77f24549,2024-12-23T13:33:50.776573,"What is Bel, and when and where was it written?","[0.626276969909668, 0.304909348487854, -3.3820...",None,"[(system, You are an expert Q&A system that is...",Bel is a new Lisp that was written by Paul Gr...,[b6f6b4daf8a1f32f7d4dd3032ef0c7b0b362c8c2481eb...,"[0.5939139614813258, 0.44074323558948497]"
